In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE131282"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE131282"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE131282.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE131282.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE131282.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyzing the dataset based on the background information and sample characteristics

# 1. Gene Expression Data Availability
# Based on the title and information, this appears to be gene expression data from brain tissue
# of multiple sclerosis patients and controls, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# The trait (Multiple Sclerosis) information can be found in key 6 'tissue' or key 4 'ms type'
# Using key 6 as it distinguishes between Grey Matter Lesion (MS) and Grey Matter (likely controls)
trait_row = 6

# Age information is in key 2 'age at death'
age_row = 2

# Gender information is in key 1 'Sex'
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert tissue type to binary MS status (1 for MS, 0 for control)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Grey Matter Lesion indicates MS, Grey Matter indicates control
    if value == 'Grey Matter Lesion':
        return 1  # MS
    elif value == 'Grey Matter':
        return 0  # Control
    else:
        return None  # Unknown or not relevant

def convert_age(value):
    """Convert age at death to continuous value"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to integer
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert sex to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.upper() == 'F':
        return 0  # Female
    elif value.upper() == 'M':
        return 1  # Male
    else:
        return None  # Unknown

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We need to create a clinical data DataFrame from the sample characteristics
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    sample_characteristics = {
        0: ['patient id: M02', 'patient id: M60', 'patient id: M44', 'patient id: M13', 'patient id: M53', 'patient id: M14', 'patient id: M56', 'patient id: M46', 'patient id: M61', 'patient id: M42', 'patient id: M28', 'patient id: M23', 'patient id: M52', 'patient id: M12', 'patient id: M32', 'patient id: M06', 'patient id: M01', 'patient id: M36', 'patient id: M59', 'patient id: M34', 'patient id: M26', 'patient id: M03', 'patient id: M54', 'patient id: M30', 'patient id: M57', 'patient id: M43', 'patient id: M48', 'patient id: M51', 'patient id: M10', 'patient id: M24'],
        1: ['Sex: F', 'Sex: M'],
        2: ['age at death: 58', 'age at death: 59', 'age at death: 80', 'age at death: 63', 'age at death: 47', 'age at death: 78', 'age at death: 88', 'age at death: 45', 'age at death: 61', 'age at death: 50', 'age at death: 54', 'age at death: 69', 'age at death: 39', 'age at death: 56', 'age at death: 44', 'age at death: 42', 'age at death: 92', 'age at death: 71', 'age at death: 77', 'age at death: 34', 'age at death: 49', 'age at death: 70', 'age at death: 35', 'age at death: 84', 'age at death: 75', 'age at death: 38', 'age at death: 64', 'age at death: 95', 'age at death: 60', 'age at death: 51'],
        6: ['tissue: Grey Matter Lesion', 'tissue: Grey Matter', 'disease duration: 21', 'disease duration: 54', 'disease duration: 31', 'disease duration: ?', 'disease duration: 36', 'disease duration: 34', 'disease duration: 17', 'disease duration: 4', 'disease duration: 42', 'disease duration: 47', 'disease duration: 16', 'disease duration: 38', 'disease duration: 30', 'disease duration: 26', 'disease duration: 33', 'disease duration: 27', 'disease duration: 41', 'disease duration: 22', 'disease duration: 6', 'disease duration: 18']
    }
    
    # Convert to DataFrame with column names matching the row indices
    clinical_data = pd.DataFrame(sample_characteristics)
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical)
    print("Preview of extracted clinical data:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement a properly formatted solution for this step that explores the dataset and extracts clinical features appropriately.

```python
import pandas as pd
import numpy as np
import json
import os
import re

# First, let's explore the directory contents to understand what data is available
print(f"Checking contents of: {in_cohort_dir}")
if os.path.exists(in_cohort_dir):
    files = os.listdir(in_cohort_dir)
    print(f"Available files: {files}")
else:
    print(f"Directory {in_cohort_dir} does not exist")

# Try loading the matrix file, which typically contains both gene expression and clinical data
matrix_path = os.path.join(in_cohort_dir, "matrix.tsv")
if os.path.exists(matrix_path):
    print(f"Found matrix file at: {matrix_path}")
    matrix_df = pd.read_csv(matrix_path, sep="\t", index_col=0)
    print(f"Matrix shape: {matrix_df.shape}")
    print("First few rows and columns of the matrix:")
    print(matrix_df.iloc[:5, :5])

    # Check if the first row contains clinical information
    if not pd.api.types.is_numeric_dtype(matrix_df.iloc[0]):
        # The first rows might contain clinical information
        # Extract these rows as clinical data
        clinical_data = matrix_df.iloc[:20].T  # Transpose to get samples as rows
        print("Potential clinical data (first 20 rows):")
        print(clinical_data.head())
        
        # Rest of the matrix is likely gene expression data
        gene_data = matrix_df.iloc[20:].T  # Transpose to get samples as rows
        print(f"Potential gene expression data shape: {gene_data.shape}")
    else:
        # The matrix might be pure gene expression without clinical data
        print("The matrix appears to contain only gene expression data.")
        gene_data = matrix_df.T  # Transpose to get samples as rows
        clinical_data = None
        
else:
    print(f"Matrix file not found at: {matrix_path}")
    # Try alternative formats
    series_matrix_path = os.path.join(in_cohort_dir, "series_matrix.txt")
    if os.path.exists(series_matrix_path):
        print(f"Found series matrix file at: {series_matrix_path}")
        # Series matrix files typically have a specific format with !Series and !Sample lines
        with open(series_matrix_path, 'r') as f:
            lines = f.readlines()
            
        # Extract clinical information from series matrix
        sample_char_dict = {}
        sample_index = None
        current_feature_idx = 0
        
        for i, line in enumerate(lines):
            if line.startswith('!Sample_geo_accession'):
                sample_index = i
                headers = line.strip().split('\t')[1:]
            elif sample_index is not None and i > sample_index and line.startswith('!Sample_'):
                feature_name = line.split('\t')[0].replace('!Sample_', '')
                values = line.strip().split('\t')[1:]
                sample_char_dict[str(current_feature_idx)] = values
                current_feature_idx += 1
                
        # Convert to DataFrame for easier processing
        if sample_index is not None:
            clinical_data = pd.DataFrame(sample_char_dict)
            clinical_data.index = headers
            print("Clinical data extracted from series matrix:")
            print(clinical_data.head())
        else:
            clinical_data = None
            print("No clinical data found in series matrix.")
    else:
        print(f"Series matrix file not found at: {series_matrix_path}")
        clinical_data = None

# 1. Gene Expression Data Availability
# Check if gene expression data is likely available based on files or matrix content
is_gene_available = False
if 'gene_data' in locals() and gene_data is not None and gene_data.shape[1] > 0:
    is_gene_available = True
    print(f"Gene expression data appears to be available (shape: {gene_data.shape})")
else:
    # Look for other indicators of gene expression data
    if 'files' in locals():
        for file in files:
            if any(term in file.lower() for term in ['gene', 'expr', 'matrix', 'platform']):
                is_gene_available = True
                print(f"Gene expression data likely available based on file: {file}")
                break

# 2. Variable Availability and Data Type Conversion
# Initialize variables
trait_row = None
age_row = None
gender_row = None

# Function to explore unique values in clinical data
def explore_clinical_data(clinical_df):
    if clinical_df is None:
        return {}
    
    feature_dict = {}
    for col in clinical_df.columns:
        unique_values = clinical_df[col].unique()
        if len(unique_values) <= 10:  # Only show if reasonable number of unique values
            feature_dict[col] = list(unique_values)
        else:
            feature_dict[col] = f"{len(unique_values)} unique values"
    return feature_dict

if clinical_data is not None:
    clinical_features = explore_clinical_data(clinical_data)
    print("\nClinical features available:")
    for col, values in clinical_features.items():
        print(f"Column {col}: {values}")

    # Analysis for trait (Multiple sclerosis)
    for col in clinical_data.columns:
        values = clinical_data[col].astype(str).str.lower()
        if any(term in ' '.join(values) for term in ['ms', 'sclerosis', 'control', 'disease', 'diagnosis', 'patient']):
            unique_vals = clinical_data[col].unique()
            if len(unique_vals) > 1:  # Not a constant feature
                trait_row = int(col)
                print(f"\nTrait data found in column {trait_row}: {list(unique_vals)}")
                break

    # Analysis for age
    for col in clinical_data.columns:
        values = clinical_data[col].astype(str).str.lower()
        if any(term in ' '.join(values) for term in ['age', 'years', 'yr']):
            unique_vals = clinical_data[col].unique()
            if len(unique_vals) > 1:  # Not a constant feature
                age_row = int(col)
                print(f"\nAge data found in column {age_row}: {list(unique_vals)[:5]}")
                break

    # Analysis for gender
    for col in clinical_data.columns:
        values = clinical_data[col].astype(str).str.lower()
        if any(term in ' '.join(values) for term in ['gender', 'sex', 'male', 'female']):
            unique_vals = clinical_data[col].unique()
            if len(unique_vals) > 1:  # Not a constant feature
                gender_row = int(col)
                print(f"\nGender data found in column {gender_row}: {list(unique_vals)}")
                break

# Define conversion functions
def convert_trait(value):
    if value is None:
        return None
    
    value_str = str(value).lower()
    # Extract value after colon if present
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    # Convert to binary: 1 for MS, 0 for Control
    if any(kw in value_str for kw in ["ms", "multiple sclerosis", "patient", "case", "affected"]):
        return 1
    elif any(kw in value_str for kw in ["control", "healthy", "normal", "hc", "unaffected"]):
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    
    value_str = str(value)
    # Extract value after colon if present
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    #


### Step 4: Gene Data Extraction

In [ ]:
# 1. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 3. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers, I can see they are not standard human gene symbols, but rather Illumina array probe IDs
# (identifiers starting with ILMN_). These are specific to Illumina microarray platforms and need to be mapped to 
# actual human gene symbols for better biological interpretation.

# Illumina probe IDs like ILMN_1343048 are specific identifiers for probes on Illumina microarray platforms
# They need to be mapped to official gene symbols or Entrez Gene IDs for cross-platform comparability

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
# From the preview, we can see that 'ID' column contains probe IDs (ILMN_*) matching our expression data
# and 'Symbol' column contains gene symbols
prob_col = 'ID'  # Column with probe IDs
gene_col = 'Symbol'  # Column with gene symbols

# 2. Get a gene mapping dataframe from the gene annotation data
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This will handle the many-to-many relationship between probes and genes as specified
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and aggregate rows with the same symbol
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene expression data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")